In [20]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from collections import OrderedDict

# Inference and learning final project

## Introduction


## Imports

In [24]:
class D_Conv(torch.nn.Module):
    def __init__(self, base_channel_size, local = False, FC = False) -> None:
        alpha = base_channel_size
        super().__init__()
        modules = OrderedDict()
        size_conv = [
            (3,0),
            (alpha,1),
            (2*alpha,2),
            (2*alpha,1),
            (4*alpha,2),
            (4*alpha,1),
            (8*alpha,2),
            (8*alpha,1),
            (16*alpha,2),
            (64*alpha,2)]
        for i in range(1,len(size_conv)):
            modules[" {} - 3*3 Conv({},{})".format(i, size_conv[i][0], size_conv[i][1])] = torch.nn.Conv2d(
                    in_channels= size_conv[i-1][0],
                    out_channels=size_conv[i][0],
                    kernel_size=3,
                    stride=size_conv[i][1])
            modules[" {} - BatchNorm()".format(i)] = torch.nn.BatchNorm2d(size_conv[i][0])
            modules[" {} - ReLU()".format(i)] = torch.nn.ReLU()
            
        self.conv = torch.nn.Sequential(modules)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16*alpha*4*4, 64*alpha),
            torch.nn.BatchNorm1d(64*alpha),
            torch.nn.ReLU(),
            torch.nn.Linear(64*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class S_Conv(torch.nn.Module):
    def __init__(self, base_channel_size, local = False, FC = False) -> None:
        alpha = base_channel_size
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(
                in_channels=3,
                out_channels= alpha,
                kernel_size=9,
                stride = 2),
            torch.nn.BatchNorm2d(alpha),
            torch.nn.ReLU()
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(alpha*4*4, 24*alpha),
            torch.nn.BatchNorm1d(24*alpha),
            torch.nn.ReLU(),
            torch.nn.Linear(24*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    

In [30]:
test = D_Conv(5)
for name, param in test.named_parameters():
    if param.requires_grad:
        print(name, param.size())

conv. 1 - 3*3 Conv(5,1).weight torch.Size([5, 3, 3, 3])
conv. 1 - 3*3 Conv(5,1).bias torch.Size([5])
conv. 1 - BatchNorm().weight torch.Size([5])
conv. 1 - BatchNorm().bias torch.Size([5])
conv. 2 - 3*3 Conv(10,2).weight torch.Size([10, 5, 3, 3])
conv. 2 - 3*3 Conv(10,2).bias torch.Size([10])
conv. 2 - BatchNorm().weight torch.Size([10])
conv. 2 - BatchNorm().bias torch.Size([10])
conv. 3 - 3*3 Conv(10,1).weight torch.Size([10, 10, 3, 3])
conv. 3 - 3*3 Conv(10,1).bias torch.Size([10])
conv. 3 - BatchNorm().weight torch.Size([10])
conv. 3 - BatchNorm().bias torch.Size([10])
conv. 4 - 3*3 Conv(20,2).weight torch.Size([20, 10, 3, 3])
conv. 4 - 3*3 Conv(20,2).bias torch.Size([20])
conv. 4 - BatchNorm().weight torch.Size([20])
conv. 4 - BatchNorm().bias torch.Size([20])
conv. 5 - 3*3 Conv(20,1).weight torch.Size([20, 20, 3, 3])
conv. 5 - 3*3 Conv(20,1).bias torch.Size([20])
conv. 5 - BatchNorm().weight torch.Size([20])
conv. 5 - BatchNorm().bias torch.Size([20])
conv. 6 - 3*3 Conv(40,2).wei